# Importação de bibliotecas

In [1]:
import pandas as pd
import numpy as np

# Recuperação dos dados

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Desenvolvimento e teste de soluções

In [4]:
# Importação do classificador
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [5]:
# Parametrização do modedlo
modelo_floresta = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=0)

In [6]:
# Variáveis que serão utilizadas
variaveis = ['Sexo_binario', 'Age']

In [7]:
# Checagem de variável
train['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [8]:
# Função para transformar variável nominal em número
def transformar_sexo(valor):
    if valor == 'female':
        return 1
    else:
        return 0

train['Sexo_binario'] = train['Sex'].map(transformar_sexo)

In [9]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sexo_binario
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [10]:
# Recuperação apenas dos dados que serão utilizados
X = train[variaveis]
y = train['Survived']

In [11]:
# Preenchimento de dados nulos
X = X.fillna(-1)

In [21]:
# Separação dos dados de treinamento em treinamento e validação
np.random.seed(1)
X_treino, X_valid, y_treino, y_valid = train_test_split(X, y, test_size=0.5)

In [22]:
# Treinamento do modelo com dados de treino
modelo_floresta.fit(X_treino,y_treino)

RandomForestClassifier(n_jobs=1, random_state=0)

In [23]:
# Previsão do modelo em dados de validação
p = modelo_floresta.predict(X_valid)

In [24]:
# Checa a acurácia
np.mean(y_valid == p)

0.7466367713004485

### Validação Cruzada

In [25]:
# Importação do Kfold
from sklearn.model_selection import KFold

In [35]:
## Exemplo e teste do Kfold 
X_falso = list(range(10))

kf = KFold(3, shuffle=True, random_state=0)
for linhas_treino, linhas_valid in kf.split(X_falso):
    print("Treino: ", linhas_treino)
    print("Validação: ", linhas_valid)
    print()

Treino:  [0 1 3 5 6 7]
Validação:  [2 4 8 9]

Treino:  [0 2 3 4 5 8 9]
Validação:  [1 6 7]

Treino:  [1 2 4 6 7 8 9]
Validação:  [0 3 5]



In [37]:
# Execução real com validação cruzada
kf = KFold(3, shuffle=True, random_state=0)

resultados = []
for linhas_treino, linhas_valid in kf.split(X):
    print("Treino: ", linhas_treino.shape[0])
    print("Validação: ", linhas_valid.shape[0])

    # Separa o X e y de treinamento e validação
    X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
    y_treino, y_valid= y.iloc[linhas_treino], y.iloc[linhas_valid]

    # Treina o modelo com os dados destacados para treino na execução
    modelo_floresta = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=0)
    modelo_floresta.fit(X_treino,y_treino)

    p = modelo_floresta.predict(X_valid)
    acc = np.mean(y_valid == p)
    resultados.append(acc)
    print("Acc: ", acc)
    print()

Treino:  594
Validação:  297
Acc:  0.7609427609427609

Treino:  594
Validação:  297
Acc:  0.7845117845117845

Treino:  594
Validação:  297
Acc:  0.7643097643097643



In [38]:
# Média final para 3 folds
np.mean(resultados)

0.7699214365881032

In [39]:
# Execução real com validação cruzada
kf = KFold(10, shuffle=True, random_state=0)

resultados = []
for linhas_treino, linhas_valid in kf.split(X):
    print("Treino: ", linhas_treino.shape[0])
    print("Validação: ", linhas_valid.shape[0])

    # Separa o X e y de treinamento e validação
    X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
    y_treino, y_valid= y.iloc[linhas_treino], y.iloc[linhas_valid]

    # Treina o modelo com os dados destacados para treino na execução
    modelo_floresta = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=0)
    modelo_floresta.fit(X_treino,y_treino)

    p = modelo_floresta.predict(X_valid)
    acc = np.mean(y_valid == p)
    resultados.append(acc)
    print("Acc: ", acc)
    print()

Treino:  801
Validação:  90
Acc:  0.7777777777777778

Treino:  802
Validação:  89
Acc:  0.797752808988764

Treino:  802
Validação:  89
Acc:  0.7078651685393258

Treino:  802
Validação:  89
Acc:  0.7303370786516854

Treino:  802
Validação:  89
Acc:  0.7865168539325843

Treino:  802
Validação:  89
Acc:  0.8089887640449438

Treino:  802
Validação:  89
Acc:  0.7528089887640449

Treino:  802
Validação:  89
Acc:  0.8089887640449438

Treino:  802
Validação:  89
Acc:  0.7752808988764045

Treino:  802
Validação:  89
Acc:  0.8089887640449438



In [40]:
# Média final para 10 folds
np.mean(resultados)

0.7755305867665419

In [42]:
## Várias repetições com 2 folds mudando o random state
# Execução real com validação cruzada
resultados = []
for rep in range(10):
    print("Repetição: ", rep)
    kf = KFold(2, shuffle=True, random_state=rep)

    for linhas_treino, linhas_valid in kf.split(X):
        print("Treino: ", linhas_treino.shape[0])
        print("Validação: ", linhas_valid.shape[0])

        # Separa o X e y de treinamento e validação
        X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
        y_treino, y_valid= y.iloc[linhas_treino], y.iloc[linhas_valid]

        # Treina o modelo com os dados destacados para treino na execução
        modelo_floresta = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=0)
        modelo_floresta.fit(X_treino,y_treino)

        p = modelo_floresta.predict(X_valid)
        acc = np.mean(y_valid == p)
        resultados.append(acc)
        print("Acc: ", acc)
        print()

Repetição:  0
Treino:  445
Validação:  446
Acc:  0.7713004484304933

Treino:  446
Validação:  445
Acc:  0.7797752808988764

Repetição:  1
Treino:  445
Validação:  446
Acc:  0.7443946188340808

Treino:  446
Validação:  445
Acc:  0.7955056179775281

Repetição:  2
Treino:  445
Validação:  446
Acc:  0.7757847533632287

Treino:  446
Validação:  445
Acc:  0.7887640449438202

Repetição:  3
Treino:  445
Validação:  446
Acc:  0.7533632286995515

Treino:  446
Validação:  445
Acc:  0.7573033707865169

Repetição:  4
Treino:  445
Validação:  446
Acc:  0.7354260089686099

Treino:  446
Validação:  445
Acc:  0.7415730337078652

Repetição:  5
Treino:  445
Validação:  446
Acc:  0.7219730941704036

Treino:  446
Validação:  445
Acc:  0.7056179775280899

Repetição:  6
Treino:  445
Validação:  446
Acc:  0.7757847533632287

Treino:  446
Validação:  445
Acc:  0.7303370786516854

Repetição:  7
Treino:  445
Validação:  446
Acc:  0.7040358744394619

Treino:  446
Validação:  445
Acc:  0.7348314606741573

Repetiçã

In [43]:
# Média final para 2 folds com alterações no random state
np.mean(resultados)

0.7550007557817302